# Reading Function File

In [1]:
# exec(open("/Users/suneetabraham/Downloads/IP_Meijer_Functions.ipynb").read())
%run ./IP_Meijer_Functions.ipynb

/Users/suneetabraham/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


New44


In [2]:
### Creating the file path for the files 
image_fp='/Users/suneetabraham/Desktop/Meijer_Project/data/UPCImages.csv'
prod_fp='/Users/suneetabraham/Desktop/Meijer_Project/data/UPC_Data.csv'

# Reading Datasets

In [4]:
### Reading the files from files paths
image_df=pd.read_csv(image_fp)
prod_df = pd.read_csv(prod_fp)

In [5]:
### Removing rows with no photos
image_df=image_df.dropna().drop_duplicates()

# Exploratory Data Analysis

In [7]:
### Creating the flag for photo with static or non static images 
image_df['url_check']=image_df['ItemDocumentValue'].apply(static_image_check)

#### We see that there is 32% images that are not static

In [8]:
#### Creating the dataset for running the code on (For ease of running)
prod_filtered=prod_df[
              (prod_df['MarketingDetails'].notna())&
              (prod_df['MarketingName'].notna())&
              (prod_df['MarketingFeatures'].notna())&
            ((prod_df['UPCTypeName']== 'UPCA')|(prod_df['UPCTypeName']== 'EAN'))]

In [9]:
### Removing the non static images from the data set 
image_df_filtered=image_df[(image_df['url_check']==True)]


### Joining Image data with the UPC data 

In [10]:
joined_data= prod_df.merge(image_df_filtered.dropna(),on='ItemSku').drop_duplicates()

In [11]:
joined_data.columns

Index(['ItemSku', 'UPCTypeName', 'ItemPrimaryIndicator', 'ProductStatusnote',
       'ProductName', 'MarketingName', 'MarketingDetails', 'MarketingFeatures',
       'L6_Name', 'L5_Name', 'L4_Name', 'L3_Name', 'L2_Name', 'L6_Code',
       'L5_Code', 'L4_Code', 'L3_Code', 'L2_Code', 'UpdatedDate',
       'ItemDocumentNote', 'ItemImageApprovalDate', 'ItemImageExpirationDate',
       'ItemImageActiveDate', 'ItemDocumentValue', 'url_check'],
      dtype='object')

# Keeping only Grocery

In [12]:
joined_data=joined_data[(joined_data['L6_Name']=='GROCERY')].dropna()

In [13]:
joined_data[['L6_Name','ItemSku']].groupby(['L6_Name']).nunique()

,ItemSku
L6_Name,
GROCERY,18152


In [14]:
joined_data=joined_data.replace(r'^\s*$', np.nan, regex=True)
joined_data.fillna({'Product_Description':'NA','MarketingDetails':'NA'}, inplace=True)

## Running Text Scoring on Meijer Existing Product Descriptions

In [16]:
joined_data['MarketingDetails']=joined_data['MarketingDetails'].apply(cleaning)

Readability

In [17]:
joined_data['readability_temp']=joined_data['MarketingDetails'].apply(flesch_kincaid_grade)
joined_data.loc[(joined_data['readability_temp']<10),'readability_score']=1
joined_data['readability_score'].fillna(0, inplace=True)
joined_data=joined_data.drop(['readability_temp'],axis=1)

Grammer

In [18]:
joined_data['grammar_score_temp'] = joined_data['MarketingDetails'].apply(errors)
joined_data['length']=joined_data['MarketingDetails'].apply(length)
joined_data['grammar_score']= 1-(joined_data['grammar_score_temp']/joined_data['length'])
joined_data=joined_data.drop(['grammar_score_temp'],axis=1)

Relevance

In [19]:
joined_data['relevance_score'] = fuzz.token_set_ratio(joined_data['MarketingDetails'],joined_data['MarketingName'])/100

Sentiment

In [20]:
joined_data['sentiment_score'] = joined_data['MarketingDetails'].apply(sent)

Product Description Score

In [21]:
relevance_wt = 0.30
grammar_wt = 0.30
readability_wt = 0.30
sentiment_wt = 0.10
joined_data['temp_score'] = (relevance_wt * joined_data['relevance_score'] +
                        readability_wt * joined_data['readability_score'] +
                        sentiment_wt * joined_data['sentiment_score'] +
                        grammar_wt * joined_data['grammar_score'])

In [22]:
joined_data.loc[(joined_data['length']<50),'Meijer_score']= joined_data['temp_score']/2
joined_data.loc[(joined_data['length']>=50),'Meijer_score']= joined_data['temp_score']

In [23]:
joined_data=joined_data.drop(['relevance_score'],axis=1)
joined_data=joined_data.drop(['readability_score'],axis=1)
joined_data=joined_data.drop(['sentiment_score'],axis=1)
joined_data=joined_data.drop(['grammar_score'],axis=1)
joined_data=joined_data.drop(['length'],axis=1)

### Extracting text from images for scores less than 0.5

In [27]:
joined_data.loc[(joined_data['Meijer_score']> 0.50), 'Text'] = joined_data['MarketingDetails']
joined_data.loc[(joined_data['Meijer_score']<=0.50), 'Text'] = joined_data['ItemDocumentValue'].apply(capture_text)

Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic
Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic
Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic


Error: Error occurred in request., ReadTimeout: HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)
Error: Operation returned an invalid status code 'Bad Request'


Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic
Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic


Error: Operation returned an invalid status code 'Bad Request'


Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic
Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic
Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='ipprojectmeijer1.cognitiveservices.azure.com', port=443): Read timed out. (read timeout=100)")': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic
Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) aft

...

Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code

Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic


Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code

Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))': /vision/v3.2/read/analyze?model-version=latest&readingOrder=basic


Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code 'PermissionDenied'
Error: Operation returned an invalid status code

### Making a half base

In [30]:
joined_data.to_csv('mid_stop.csv')

In [31]:
### adding the name of product into the text column 
joined_data['Text']='Product Name:'+joined_data['MarketingName']+'  '+ joined_data['Text']
### Creating the data frame for getting product_description 
prod_desc_df=joined_data[['ItemSku','Text']]

In [32]:
# Group the dataframe by the ItemSku column and concatenate the text column for each group
result = prod_desc_df.dropna().groupby(['ItemSku']).apply(lambda x: ' '.join(x['Text']))
# # Reset the index and rename the column to get the desired output
result = result.reset_index().rename(columns={0: 'Text'})

,ItemSku,Text
0,45019300,Product Name:Lactaid Vanilla Ice Cream 32 oz ...
1,45019302,Product Name:Lactaid Chocolate Ice Cream Quart...
2,60916400,Product Name:2.5 oz. Semi-Solid Milk Chocolate...
3,60916700,Product Name:8 oz. Milk Chocolate Christmas Tr...
4,285004018,Product Name:Lucky Leaf Key Lime Filling & Top...
...,...,...
17046,763047789073,Product Name:STARBUCKS® by Nespresso® Vertuo L...
17047,763047789077,Product Name:STARBUCKS® by Nespresso® Vertuo L...
17048,763047789081,Product Name:STARBUCKS® by Nespresso® Vertuo L...
17049,763047789085,Product Name:STARBUCKS® by Nespresso® Vertuo L...


In [35]:
# ### Running the product_description function for text captured column
result['Product_Description']=result['Text'].apply(product_description)

In [37]:
### Apply the replace function to the Product Description column
result['Product_Description'] = result['Product_Description'].apply(lambda x: x.replace('\n', ''))

In [38]:
### Joining the data set back to original data set
result_df= joined_data.merge(result.dropna(),on='ItemSku').drop_duplicates()

In [39]:
result_df=result_df.replace(r'^\s*$', np.nan, regex=True)
result_df.fillna({'Product_Description':'NA','MarketingDetails':'NA'}, inplace=True)

In [40]:
result_df['Product_Description']=result_df['Product_Description'].apply(cleaning)

### Running Scoring Algorithm

Readability

In [41]:
result_df['readability_temp']=result_df['MarketingDetails'].apply(flesch_kincaid_grade)
result_df.loc[(result_df['readability_temp']<10),'readability_score']=1
result_df['readability_score'].fillna(0, inplace=True)
result_df=result_df.drop(['readability_temp'],axis=1)

Grammer

In [42]:
result_df['grammar_score_temp'] = result_df['MarketingDetails'].apply(errors)
result_df['length']=result_df['MarketingDetails'].apply(length)
result_df['grammar_score']= 1-(result_df['grammar_score_temp']/result_df['length'])
result_df=result_df.drop(['grammar_score_temp'],axis=1)

Relevance

In [43]:
result_df['relevance_score'] = fuzz.token_set_ratio(result_df['MarketingDetails'], result_df['MarketingName'])/100


Sentiment 

In [44]:
result_df['sentiment_score'] = result_df['MarketingDetails'].apply(sent)

In [45]:
relevance_wt = 0.30
grammar_wt = 0.30
readability_wt = 0.30
sentiment_wt = 0.10
result_df['temp_score'] = (relevance_wt * result_df['relevance_score'] +
                        readability_wt * result_df['readability_score'] +
                        sentiment_wt * result_df['sentiment_score'] +
                        grammar_wt * result_df['grammar_score'])

In [46]:
result_df.loc[(joined_data['length']<50),'ChatGPT_Score']= result_df['temp_score']/2
result_df.loc[(joined_data['length']>=50),'ChatGPT_Score']= result_df['temp_score']

In [47]:
result_df=result_df.drop(['relevance_score'],axis=1)
result_df=result_df.drop(['readability_score'],axis=1)
result_df=result_df.drop(['sentiment_score'],axis=1)
result_df=result_df.drop(['grammar_score'],axis=1)
result_df=result_df.drop(['length'],axis=1)

In [48]:
result_df.to_excel('FinalData_Set.xlsx')